# ESP 离线语音识别步骤：

### 1  语音采集并转换成频谱图

### 2  利用TensorFlow机器学习对频谱图进行分类

### 3  下载针对ESP32的tflite,编译运行

## 设置模型保存地址

In [26]:
# Define paths to model files
import os
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'model'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'model_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'model.tflite'
MODEL_TFLITE_MICRO = MODELS_DIR + 'model.cc'

## **准备基础库**

In [27]:
import tensorflow as tf
from tensorflow.io import gfile
import tensorflow_io as tfio
from tensorflow.python.ops import gen_audio_ops as audio_ops
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.data import Dataset

from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm
# from collections import Counter 
# import datetime
# Pandas is a data manipulation library 
import pandas as pd
# Matplotlib is a graphing library
import matplotlib.pyplot as plt
# Math is Python's math library
import math

import pyaudio
import wave
import sys
import numpy as np

import time

from pydub import AudioSegment


from playsound import playsound


## **基础函数工具准备**

In [28]:
background_files = gfile.glob( 'speech_commands_v0.01/_background_noise_/*.wav') 
words_negative = [
    "backward",
#     'bed',
    'noise',
#     'bird',
#     'cat',
#     'dog',
#     'down',
#     'eight',
#     'five',
#     'follow',
#     'forward',
#     'four',
#     'go',
#     'house',
#     'learn',
#     'left',
#     'marvin',
#     'nine',
#     'no',
#     'off',
#     'on',
#     'one',
#     'right',
#     'seven',
#     'sheila',
#     'six',
#     'stop',
#     'three',
#     'tree',
#     'two',
#     'up',
#     'visual',
#     'wow',
#     'yes',
#     "zero"
]
words_positive =[
    'xiangzhi',
    'jizhang',
   
#     "backward",
#     'bed',
#     'bird',
#     'cat',
#     'dog',
#     'down',
#     'eight',
#     'five',
#     'follow',
#     'forward',
#     'four',
#     'go',
#     'house',
#     'learn',
#     'left',
#     'marvin',
#     'nine',
#     'no',
#     'off',
#     'on',
#     'one',
#     'right',
#     'seven',
#     'sheila',
#     'six',
#     'stop',
#     'three',
#     'tree',
#     'two',
#     'up',
#     'visual',
#     'wow',
#     'yes',
#     "zero"
]

# 归一处理(-1,1)
def normalize(audio_tensor_slice):
  audio = tf.cast(audio_tensor_slice,tf.float32) #转float32，用于计算
  audio = audio - np.mean(audio)
  audio = audio / np.max(np.abs(audio))
  return audio

# 随机左右移动音频
def roll_audio(audio_normalized):
  voice_start, voice_end = tfio.audio.trim(audio_normalized, axis=0, epsilon=0.1) 
  end_gap=len(audio_normalized) - voice_end
  random_offset = int(np.random.uniform(0, voice_start+end_gap))
  return np.roll(audio_normalized,-random_offset+end_gap)

# 添加噪音
def add_noise(audio):
  background_volume = np.random.uniform(0, 0.3)
  return audio + background_volume * generate_noise()

# 生成噪音
def generate_noise():
  background_file = np.random.choice(background_files)
  background_tensor = tfio.audio.AudioIOTensor(background_file)
  background_start = np.random.randint(0, len(background_tensor) - 16000)
  background = normalize( background_tensor[background_start:background_start+16000] )
  return background

#########################################################################################
# 搜索负样本文件名
def collect_neg_sample_file(words_negative):
  file_names = []
  for word in tqdm(words_negative, desc="collecting negative samples"):
    for file_name in tqdm(gfile.glob('speech_commands_v0.01/'+word+'/*.wav'), desc="checking word "+word, leave=False):
      audio_tensor = tfio.audio.AudioIOTensor(file_name)

      #检查文件长度是否正确？
      if audio_tensor.shape[0] != 16000 :
        continue
      #检查有效声音长度是否足够
      audio = normalize(audio_tensor[:])
      position = tfio.audio.trim(audio, axis=0, epsilon=0.1)
      if int(position[1] - position[0]) < 16000/4:
        continue
      file_names.append([file_name,0])
  for i in range(2000):
    file_names.append(["silence",0])
  return file_names

# 负样本预处理
def preprocess_neg_samples(neg_file_names):
  spectrograms = []
  for file_name in tqdm(neg_file_names, desc="preprocessing negative samples", leave=False):
    if file_name == "silence":
      audio = generate_noise()
    else:
      audio_tensor = tfio.audio.AudioIOTensor(file_name)
      audio = normalize(audio_tensor[:])
      audio = roll_audio(audio)
      audio = add_noise(audio)

    spectrogram = get_spectrogram(audio)
    spectrograms.append(spectrogram)
  return spectrograms


# 搜索正样本文件名
def collect_posi_sample_file(words_positive):
  file_names = []
  for word in tqdm(words_positive, desc="collecting words_positive samples"):
    for file_name in tqdm(gfile.glob('speech_commands_v0.01/'+word+'/*.wav'), desc="checking word "+word, leave=False):
      file_names.append([file_name,words_positive.index(word)+1])
  return file_names
# 正样本预处理
def preprocess_posi_samples(posi_file_names):
  spectrograms = []
  for file_name in tqdm(posi_file_names, desc="preprocessing positive samples", leave=False):
    audio_tensor = tfio.audio.AudioIOTensor(file_name)
    #检查文件长度是否正确？
    if audio_tensor.shape[0] > 16000 :
      if np.random.uniform(0, 1) > 0.5:
        audio = audio_tensor[:16000]
      else:
        audio = audio_tensor[audio_tensor.shape[0]-16000:]
    elif audio_tensor.shape[0] < 16000 :
      audio_zero = tf.zeros([16000-audio_tensor.shape[0],1],dtype=tf.int16)
      audio = tf.concat([audio_tensor[:],audio_zero],axis=0)
    else:
      audio = audio_tensor[:]

    audio = normalize(audio)
    audio = roll_audio(audio) 
    audio = add_noise(audio)
    spectrogram = get_spectrogram(audio)
    spectrograms.append(spectrogram)
  return spectrograms

#########################################################################################
# 音谱图工具
def get_spectrogram1(audio):
    #再做一次归一处理
#     audio = audio - np.mean(audio)
#     audio = audio / np.max(np.abs(audio))
    
    audio=np.array(audio)
    # create the spectrogram
    spectrogram = audio_ops.audio_spectrogram(audio,
                                              window_size=512,
                                              stride=160,
                                              magnitude_squared=True).numpy()
    # reduce the number of frequency bins in our spectrogram to a more sensible level

    spectrogram = tf.nn.pool(
        input=tf.expand_dims(spectrogram, -1),
        window_shape=[1, 6],
        strides=[1, 6],
        pooling_type='AVG',
        padding='SAME')
    spectrogram = tf.squeeze(spectrogram, axis=0)
    spectrogram = np.log10(spectrogram + 1e-6)
    return spectrogram
def get_spectrogram(audio):
    audio =np.array(tf.squeeze(audio)) 
    
    audio = audio - np.mean(audio)
    audio = audio / np.max(np.abs(audio))
    
    STRIDE=160
    FFT_INPUT_SIZE=512
    FFT_OUT_SIZE=np.round(FFT_INPUT_SIZE/2+1).astype(np.int16)

    hanning_window = np.array([0.50 - 0.5 * np.cos(2 * np.pi * n / (FFT_INPUT_SIZE - 1)) for n in range(FFT_INPUT_SIZE)])
    
    spectrogram=[]
    for i in np.arange(98):
        x=audio[i*160:i*160+FFT_INPUT_SIZE]
        
        if x.shape[0]<512:
            x=np.append(x,np.zeros(512-x.shape[0]))
        
        x=x*hanning_window
                        
        y =np.square(abs(np.fft.fft(x))) 
        y =y[0:FFT_OUT_SIZE]
        y_p=[]
        y=np.append(y,np.zeros(6-(FFT_OUT_SIZE%6)))#padding=same
        for j in np.arange(43):
            y_p.append((y[j*6]+y[j*6+1]+y[j*6+2]+y[j*6+3]+y[j*6+4]+y[j*6+5])/6)
             
        spectrogram.append(y_p)
      
    spectrogram = np.log10(np.array(spectrogram) + 1e-6)
    spectrogram = np.reshape(np.array(spectrogram),(98,43,1))
    spectrogram=spectrogram.astype(np.float32)  

    return  spectrogram



In [32]:
audio_tensor = tfio.audio.AudioIOTensor("speech_commands_v0.01/test/1640444576515.wav")
audio = audio_tensor[:]

print(np.array(audio.shape))


[16384     1]


In [30]:
audio_tensor = tfio.audio.AudioIOTensor("speech_commands_v0.01/xiangzhi/1640488607529.wav")
audio = audio_tensor[:]

# #信号形状
# plt.plot(audio)
# plt.show()

# #频谱  
# audio=np.array(audio)
# spectrogram = audio_ops.audio_spectrogram(audio,
#                                           window_size=16000,
#                                           stride=160,
#                                           magnitude_squared=True).numpy()
# # reduce the number of frequency bins in our spectrogram to a more sensible level
# spectrogram = tf.squeeze(spectrogram, axis=0)

# spectrogram=np.array(spectrogram[0])
# plt.plot(spectrogram)
# plt.show()

# #频谱图   
# audio=np.array(audio)
# # create the spectrogram
# spectrogram = audio_ops.audio_spectrogram(audio,
#                                           window_size=512,
#                                           stride=160,
#                                           magnitude_squared=True).numpy()
# # reduce the number of frequency bins in our spectrogram to a more sensible level
# spectrogram = tf.squeeze(spectrogram, axis=0)

# spectrogram =np.array(spectrogram)

# plt.matshow(spectrogram)


## ONE

## **正式开始！！！！！！（数据预处理）**

In [31]:
# 获取正样本的文件名 
posi_filenames = collect_posi_sample_file(words_positive)

collecting words_positive samples:   0%|          | 0/2 [00:00<?, ?it/s]

checking word xiangzhi:   0%|          | 0/48 [00:00<?, ?it/s]

checking word jizhang:   0%|          | 0/49 [00:00<?, ?it/s]

In [32]:
# 获取负样本的文件名 
neg_filenames = collect_neg_sample_file(words_negative)

collecting negative samples:   0%|          | 0/2 [00:00<?, ?it/s]

checking word backward: 0it [00:00, ?it/s]

checking word noise:   0%|          | 0/1100 [00:00<?, ?it/s]

In [35]:
# 平衡正负样本数据
rate = int(len(neg_filenames) / len(posi_filenames))
print("平衡前比例",rate,"(",len(neg_filenames),"/",len(posi_filenames),")")
if rate > 0:
    posi_filenames = posi_filenames*rate
print("平衡后比例",int(len(neg_filenames) / len(posi_filenames)),"(",len(neg_filenames),"/",len(posi_filenames),")")

平衡前比例 1 ( 2000 / 1940 )
平衡后比例 1 ( 2000 / 1940 )


In [36]:
# 生成负样本音谱图
spectrograms_neg = preprocess_neg_samples([i[0] for i in neg_filenames])
labels_neg=[i[1] for i in neg_filenames]

preprocessing negative samples:   0%|          | 0/2000 [00:00<?, ?it/s]

In [37]:
# 生成正样本音谱图 
spectrograms_posi = preprocess_posi_samples([i[0] for i in posi_filenames])
labels_posi=[i[1] for i in posi_filenames]

preprocessing positive samples:   0%|          | 0/1940 [00:00<?, ?it/s]

In [38]:
# 组合音谱图、标签
labels = labels_posi+labels_neg
spectrograms = spectrograms_posi + spectrograms_neg

## **训练模型**

In [39]:
# 分开训练、验证集
x_train, x_val, y_train, y_val = train_test_split(np.array(spectrograms), np.array(labels), test_size=0.2, random_state=42)

In [142]:
print(np.array(y_train)[1:30])

[2 0 2 0 1 1 0 0 2 0 0 0 1 0 2 0 0 0 0 2 0 0 0 0 2 2 2 0 0]


In [40]:
# 打包 (CPU 13mins? / GPU 10mins? )
batch_size = 30

dataset_train = Dataset.from_tensor_slices(
    (x_train, y_train)
).repeat(
    count=-1
).batch(
    batch_size
)

dataset_val = Dataset.from_tensor_slices((x_val, y_val)).batch(len(y_val))

In [41]:
# 搭建模型
model = Sequential([
    Conv2D(4, 3, 
           padding='same',
           activation='relu',
           kernel_regularizer=regularizers.l2(0.001),
           name='conv_layer1',
           input_shape=(x_train[0].shape[0], x_train[0].shape[1], 1)),
    MaxPooling2D(name='max_pooling1', pool_size=(2,2)),
    Conv2D(4,3, 
           padding='same',
           activation='relu',
           kernel_regularizer=regularizers.l2(0.001),
           name='conv_layer2'),
    MaxPooling2D(name='max_pooling2', pool_size=(2,2)),
    Flatten(),
    Dropout(0.2),
    Dense(
        32,
        activation='relu',
        kernel_regularizer=regularizers.l2(0.001),
        name='hidden_layer1'
    ),
    Dense(
        3, 
        activation='softmax',
        kernel_regularizer=regularizers.l2(0.001),
        name='output'
    )
])

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_layer1 (Conv2D)        (None, 98, 43, 4)         40        
                                                                 
 max_pooling1 (MaxPooling2D)  (None, 49, 21, 4)        0         
                                                                 
 conv_layer2 (Conv2D)        (None, 49, 21, 4)         148       
                                                                 
 max_pooling2 (MaxPooling2D)  (None, 24, 10, 4)        0         
                                                                 
 flatten (Flatten)           (None, 960)               0         
                                                                 
 dropout (Dropout)           (None, 960)               0         
                                                                 
 hidden_layer1 (Dense)       (None, 32)                3

In [43]:
# 编译模型
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
              metrics=['sparse_categorical_accuracy'])

In [44]:
# 开始训练
history = model.fit(
    dataset_train,
    steps_per_epoch=len(x_train) // batch_size,
    epochs=3,
    validation_data=dataset_val,
    validation_steps=1,
)
model.save(MODEL_TF)

Epoch 1/3
105/105 [==============================] - 16s 69ms/step - loss: 0.6941 - sparse_categorical_accuracy: 0.7022 - val_loss: 0.2999 - val_sparse_categorical_accuracy: 0.9378
Epoch 2/3
105/105 [==============================] - 7s 70ms/step - loss: 0.2653 - sparse_categorical_accuracy: 0.9362 - val_loss: 0.1633 - val_sparse_categorical_accuracy: 0.9670
Epoch 3/3
105/105 [==============================] - 7s 66ms/step - loss: 0.1915 - sparse_categorical_accuracy: 0.9568 - val_loss: 0.1432 - val_sparse_categorical_accuracy: 0.9759
INFO:tensorflow:Assets written to: models/model\assets


## **模型测试**

In [82]:
x_train, x_test, y_train, y_test = train_test_split(np.array(spectrograms), np.array(labels), test_size=0.2, random_state=12)
dataset_test = Dataset.from_tensor_slices((x_test, y_test)).batch(len(y_test))

In [83]:
print(x_test.shape)
model.predict(x_test,batch_size = 1)


(799, 98, 43, 1)


array([[2.82931251e-06, 6.61376762e-05, 9.99930978e-01, 5.31486969e-16,
        2.59403805e-12],
       [4.39871401e-05, 2.92905206e-05, 9.99926686e-01, 7.42833326e-15,
        4.18594846e-11],
       [1.76439262e-05, 1.77345501e-05, 9.99964595e-01, 2.52821102e-15,
        6.43703668e-09],
       ...,
       [1.23359205e-05, 4.76207118e-04, 1.97880006e-08, 2.53310652e-08,
        9.99511361e-01],
       [1.69168075e-07, 2.47470474e-13, 2.62105515e-15, 9.99999762e-01,
        1.03961938e-07],
       [2.88935797e-03, 4.14555871e-07, 1.12508224e-06, 9.96991158e-01,
        1.18068288e-04]], dtype=float32)

In [84]:
results = model.evaluate(dataset_test)

1/1 [==============================] - 0s 406ms/step - loss: 0.0937 - sparse_categorical_accuracy: 0.9987


In [85]:
predictions = model.predict_on_batch(np.array(x_test))
print(predictions.shape)
print(predictions[0:20])
# print(np.array(x_test).shape)
print(np.array(y_test)[0:20])

(799, 5)
[[2.8292857e-06 6.6137552e-05 9.9993098e-01 5.3148289e-16 2.5940233e-12]
 [4.3986889e-05 2.9290550e-05 9.9992669e-01 7.4282486e-15 4.1859325e-11]
 [1.7643910e-05 1.7734550e-05 9.9996459e-01 2.5282110e-15 6.4370362e-09]
 [3.2663828e-04 1.6932459e-04 9.9950409e-01 2.1795502e-13 2.3145247e-08]
 [1.0000000e+00 9.9657409e-28 3.6667993e-17 2.4166979e-12 5.5073953e-17]
 [9.9784791e-01 4.0028726e-06 2.1300630e-03 1.8002185e-05 4.8532272e-08]
 [4.6629616e-08 9.9999154e-01 4.0274563e-06 1.5938908e-11 4.3600876e-06]
 [5.2184874e-05 2.6094465e-05 9.9992168e-01 1.0431359e-13 2.5616254e-10]
 [7.4949247e-01 3.4033005e-09 1.8666645e-07 2.4979116e-01 7.1620126e-04]
 [3.5936827e-07 9.9469196e-07 1.5968477e-11 9.6416898e-08 9.9999857e-01]
 [9.9999011e-01 1.9559710e-11 1.3829992e-06 8.3427876e-06 1.3227698e-07]
 [1.0698609e-05 1.0277851e-06 1.6360737e-08 1.5948908e-07 9.9998808e-01]
 [9.9906904e-01 6.5061943e-07 5.2376831e-04 1.3955836e-05 3.9254426e-04]
 [3.8847575e-04 1.6807045e-11 3.5532062e-0

In [48]:
format(predictions[11][0], '.2f')

'0.00'

In [27]:
decision = [1 if p > 0.95 else 0 for p in predictions]
tf.math.confusion_matrix(y_test, decision)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1628,    5],
       [ 161, 1234]])>

In [52]:
model.save('my_model.h5')

In [53]:
model2 = tf.keras.models.load_model('my_model.h5')
results2 = model2.evaluate(dataset_test)

1/1 [==============================] - 5s 5s/step - loss: 0.1030 - accuracy: 0.9812


### 验证预测值

In [80]:
model2 = tf.keras.models.load_model(MODEL_TF)
# model2.summary()

In [115]:
print(x_test.shape)
t_x_1=x_test[6].reshape(1,98,43,1)
# print(np.array(tf.squeeze(t_x_1)))
gg=np.array(tf.squeeze(t_x_1))
# np.savetxt("tt.txt",gg,delimiter=",")
s1=np.array([0.092123725,-0.021073706,-0.33380485,-0.97795963,-1.8645535,-1.9453416,-1.6915576,-2.3373103,-1.6277491,-1.9380122,-2.1726384,-2.5387406,-2.7705467,-2.5460503,-2.179928,-2.4284387,-2.3985803,-2.092095,-2.051741,-2.223046,-1.9486809,-2.0572183,-2.8095288,-2.5805118,-2.5493283,-2.4962156,-2.3536105,-2.561395,-2.6021748,-2.8048239,-2.429687,-2.5395055,-2.804461,-2.5565805,-2.630026,-3.23755,-2.8350873,-2.3367758,-2.4952252,-2.2476327,-2.5064852,-2.3566432,-2.5278907,-0.49192867,-0.46092558,-0.49199703,-1.6155941,-2.433337,-1.9199797,-1.9159797,-2.0358152,-2.0316448,-1.9595017,-2.5320685,-2.5123568,-2.7132995,-2.6412451,-2.6536877,-2.3921623,-2.0941107,-2.223173,-2.106561,-2.930515,-2.0131993,-2.4707148,-2.5224264,-2.6106412,-2.6790996,-2.4633274,-2.6910417,-2.7125106,-2.4059045,-2.784523,-2.1031008,-2.8949554,-2.8296375,-2.5480628,-2.8982582,-3.0118418,-2.720387,-2.4520116,-2.2304258,-2.3359625,-2.5823078,-2.519566,-2.782962,-1.1292909,-0.7894363,-0.7719279,-1.9107662,-1.6705444,-1.6009,-1.8027962,-2.107704,-1.9648842,-1.9031246,-2.3308947,-2.243943,-2.719353,-2.5505304,-2.347135,-2.0580556,-2.2006485,-2.5421596,-2.5138068,-2.328982,-2.176289,-2.0409706,-2.410576,-2.3179007,-2.7271318,-1.9782085,-2.8390234,-2.9689274,-2.6498926,-2.7013144,-2.3025522,-2.4658377,-2.1052282,-3.0333338,-2.738333,-2.6019223,-2.297854,-2.5709288,-1.8026782,-1.9829934,-2.2618759,-2.5785985,-2.8322709,-2.1495538,-1.2753707,-1.1111239,-1.970536,-1.7642107,-1.5212498,-1.8877914,-2.3327234,-1.8588686,-2.658685,-2.4427297,-2.1086326,-2.5808966,-2.5043235,-2.7405334,-2.480614,-2.675293,-2.5307064,-2.614796,-2.4369395,-2.6795695,-2.1020226,-2.567802,-2.1277187,-2.645253,-2.291212,-2.311775,-2.4412086,-2.2026825,-2.2752645,-2.5818017,-2.3102083,-2.2339268,-2.7471762,-2.7630956,-2.5441442,-2.451598,-2.7017903,-2.187279,-1.9652307,-2.5429108,-2.3545394,-3.2955258,-1.9508815,-1.4778864,-1.405619,-2.138807,-1.513821,-1.8820734,-2.2730885,-2.2493663,-2.0650241,-3.1423275,-2.4697938,-2.1140473,-2.4077375,-2.5703926,-2.3839498,-2.4243171,-2.6194851,-2.758843,-2.7192116,-2.8385108,-2.5511043,-2.4675303,-2.603221,-1.9921799,-2.650882,-2.5095875,-2.8942046,-2.7364955,-2.2461302,-2.0554926,-2.5268059,-2.660954,-2.8812113,-2.8616896,-2.7080677,-2.8393807,-2.6368027,-2.4398801,-2.5942302,-2.1899183,-2.7942903,-2.6067636,-2.6673496,-2.1693594,-1.3224447,-1.7632692,-2.6998572,-2.1687298,-1.9642041,-2.2909422,-2.277573,-2.085079,-2.6187358,-2.3897815,-2.1571093,-2.0542734,-2.4872985,-2.3615272,-2.4489236,-2.5598881,-2.560821,-2.0879676,-2.474797,-2.4591668,-2.730904,-2.5651557,-2.3479471,-2.4723625,-2.730425,-2.6455948,-2.3687472,-2.2078657,-2.1800034,-2.6840553,-2.524774,-2.6245937,-2.7878578,-2.533467,-2.4095254,-2.565364,-2.6798317,-2.6679087,-2.0416515,-2.1518412,-2.2934518,-2.7127852,-2.2770908,-1.1465312,-1.9489688,-2.3111398,-2.5465212,-1.8419774,-2.051401,-2.761304,-1.6926154,-2.4878933,-2.136464,-2.3834708,-2.354601,-2.2609086,-2.744702,-3.161304,-2.79532,-2.3231814,-2.0365543,-2.2773309,-2.1902204,-2.367302,-2.4959018,-2.248903,-2.0507848,-2.6363208,-2.7819421,-2.390354,-2.4067388,-2.4805148,-3.129948,-2.4537206,-2.55058,-2.8529794,-2.268701,-2.1459424,-2.782677,-2.6179748,-2.2995899,-2.0629127,-2.0671992,-2.5130818,-2.299582,-2.1769664,-1.8040755,-2.3509126,-2.3086312,-1.9427327,-1.3790013,-1.9433246,-2.2538564,-1.7020506,-2.5500233,-2.1275203,-2.5679915,-2.9195483,-2.3752952,-2.523401,-3.2104287,-2.498432,-2.2764592,-2.3784127,-2.446944,-2.3333127,-2.442348,-2.6389887,-2.4670982,-2.170803,-2.5510294,-2.7051897,-2.6068609,-2.646082,-2.5293045,-2.5700314,-2.2459712,-2.072552,-2.6361592,-3.0522816,-2.691344,-2.33157,-2.0053594,-1.9484799,-1.984261,-2.7595582,-2.7137287,-2.6307104,-1.7215708,-1.5793366,-2.066758,-2.8777611,-1.7654239,-1.6039628,-1.9381005,-2.0705605,-2.0333757,-2.4280238,-2.2674956,-2.656841,-2.8944788,-2.9912167,-2.5980108,-2.7898214,-2.6171556,-2.520457,-2.143885,-2.8731956,-2.3583446,-2.997918,-2.5356152,-2.7615592,-2.4011586,-2.6060927,-3.0024881,-2.5836277,-2.748139,-2.2618482,-2.6056118,-2.479065,-2.3719847,-2.9498432,-2.8193512,-2.7834475,-2.280458,-2.4351382,-2.4507818,-2.3744698,-2.4489608,-3.0705447,-2.3448386,-1.822577,-1.1826503,-1.9440268,-2.4047282,-1.8675197,-1.7164881,-1.8385111,-2.2688668,-2.1049612,-2.1122665,-2.5112538,-2.5021267,-2.6159065,-3.350506,-2.6389203,-2.8446498,-2.4913635,-2.6601286,-2.713416,-3.112691,-2.6887224,-2.632066,-2.3517926,-2.9221444,-2.6498263,-2.5932739,-2.755763,-2.5313327,-2.6820197,-2.3598924,-2.7833178,-2.6885133,-2.6647918,-2.9583342,-2.7142723,-2.4641905,-2.255361,-2.7933776,-2.2750833,-2.2753599,-2.1266568,-2.8839839,-2.2592258,-1.8033228,-1.1809772,-1.6578091,-2.368316,-1.9531293,-2.0261424,-2.0423868,-2.2806647,-2.6656673,-2.4637709,-2.5233893,-2.2815511,-2.465603,-3.0338376,-2.6201212,-2.3841522,-2.7624571,-2.675956,-2.4873974,-2.7629607,-2.8090117,-2.4273968,-2.2227662,-2.2518723,-2.5032482,-2.298521,-2.1067371,-2.5222802,-2.7347963,-2.3740866,-2.3554204,-2.1508965,-1.9678199,-2.6789687,-2.6442454,-2.5546901,-2.3684115,-2.2825472,-1.8841674,-2.208254,-2.2839217,-2.6917405,-2.4668465,-1.7739263,-1.6143675,-1.3587875,-2.0072699,-1.8323631,-2.007949,-1.9053321,-2.617477,-2.324241,-2.4632726,-2.5297549,-2.3701973,-2.6269047,-2.2066479,-2.454556,-2.2344687,-2.5550041,-2.15797,-2.4481053,-2.3950775,-2.231452,-2.4480784,-1.8374155,-1.8068621,-2.3915565,-2.4397871,-2.4113374,-2.3100445,-2.8255284,-2.559562,-2.3681726,-2.6992924,-2.142223,-2.560802,-2.2770152,-2.447049,-2.359541,-2.6393876,-2.2241094,-2.2873285,-2.0728605,-2.6456156,-2.3445222,-2.2245917,-1.5529972,-1.7239208,-2.2478719,-1.9237067,-1.8730913,-2.2146459,-2.2760289,-2.1769624,-2.2232504,-2.4676387,-2.0322976,-2.2364986,-2.1275578,-2.4462538,-2.7333434,-2.4350457,-2.223455,-2.5978863,-2.4545586,-2.3417711,-1.8679681,-2.071095,-2.0849051,-2.2653608,-2.3488407,-2.5058541,-2.504998,-2.6048176,-3.473133,-2.755736,-2.3849313,-2.1455746,-2.1441984,-2.0477567,-2.3238277,-2.4486263,-2.298598,-2.307133,-2.314968,-2.3723881,-2.6082149,-2.6681252,-1.5988375,-1.5049293,-1.7455343,-2.218613,-2.3789485,-1.8470598,-2.7339385,-1.9006566,-2.2577243,-1.8153541,-2.5417752,-1.9932284,-2.013565,-2.0755942,-2.9752402,-2.421679,-1.8849622,-2.3996704,-2.3919516,-2.4595969,-2.3046093,-2.0527494,-2.1860363,-2.1683307,-1.859365,-2.1140924,-2.4269361,-2.2468262,-1.9184117,-2.5700357,-2.1062598,-1.8261611,-1.6268761,-1.8757646,-2.0246964,-2.1693764,-2.162823,-2.092544,-1.9560347,-2.2957027,-1.9242678,-1.7958437,-2.4020467,-1.5056633,-1.4233994,-2.1614568,-2.1374304,-2.1331913,-1.6850684,-2.6631627,-2.2032943,-2.0794764,-2.5993896,-2.3163815,-2.2617266,-2.9778411,-2.6444745,-2.9128973,-2.7499862,-2.2822044,-2.67871,-2.35354,-2.1714132,-1.8456051,-1.7349384,-2.2667327,-2.338304,-2.6787012,-2.478625,-2.497687,-2.0361676,-1.9825208,-2.4812214,-1.9840354,-2.3198357,-2.2452817,-1.8451648,-2.393826,-2.2814004,-2.4923456,-2.4414604,-2.8330233,-2.4954698,-2.1286285,-1.9233693,-2.7511468,-1.4584029,-1.6378149,-2.0081527,-2.2007728,-2.0306983,-1.5592483,-2.0665505,-2.0905864,-2.4315305,-2.454656,-2.4844496,-2.0642111,-2.4238033,-2.813508,-2.419997,-2.754797,-2.6388545,-2.8063574,-2.7379308,-2.4253669,-2.2716627,-2.5384817,-2.0875924,-2.7468054,-2.1857083,-2.2540715,-2.341997,-2.3526952,-2.362126,-2.1268358,-2.055487,-2.2736695,-2.1126904,-2.0145404,-1.9912568,-2.212315,-2.5779603,-2.7803762,-2.2831717,-2.511386,-2.1197963,-1.9001207,-3.0173512,-1.8662134,-1.9131957,-2.0689557,-1.7455337,-1.7695336,-1.3579988,-1.9523015,-2.254171,-2.2113156,-2.1193163,-2.6252427,-2.4037776,-2.0824752,-2.369374,-2.5540626,-2.4657776,-2.3844812,-2.3217623,-2.722881,-2.1867967,-2.2557445,-2.631788,-2.4214323,-2.752888,-1.9338464,-2.4616969,-2.6525626,-2.0668561,-2.1066296,-2.0955026,-2.2291381,-2.7560406,-2.190254,-1.8835844,-1.8029329,-2.5183618,-2.516896,-3.047815,-1.9127913,-2.0870955,-2.689009,-2.1341674,-2.531079,-1.7689829,-1.7890483,-2.2440913,-1.7079295,-1.7309805,-1.4894801,-1.9629658,-2.5883982,-2.376906,-2.0078883,-2.477988,-2.4156005,-2.1508296,-2.7454164,-2.629953,-2.6407788,-2.2477622,-2.0515811,-2.4261842,-2.2829304,-2.311491,-2.2915335,-2.4114013,-2.624103,-2.0770676,-2.5262194,-2.5604558,-2.4056332,-2.7565782,-2.709661,-2.4423082,-2.2240293,-2.3271208,-2.3280115,-2.6828012,-2.8593574,-2.5483942,-2.9344606,-2.1441383,-2.5743396,-2.430144,-2.3769557,-2.5497909,-1.4970983,-1.8629954,-2.3154962,-1.8147831,-1.8755517,-1.5228306,-1.9891247,-2.6723142,-2.268287,-2.4245358,-2.2738235,-2.2354205,-2.2569137,-2.3129168,-2.6323373,-2.7736514,-1.9808314,-2.3188732,-2.696865,-2.470924,-2.2227314,-2.163158,-2.3163183,-2.2693748,-2.0469108,-2.1640697,-2.7368162,-2.4410188,-2.439484,-2.256243,-2.605408,-2.1365302,-2.6907537,-2.7198668,-2.3496962,-2.3239245,-2.3072088,-2.1789255,-2.125778,-2.6725528,-2.8881457,-2.8555791,-3.2321937,-1.6101422,-2.2615,-1.7903508,-1.9134465,-2.2941837,-1.850581,-1.8219477,-2.7130141,-2.2244241,-2.5111184,-2.0873349,-2.4156806,-2.7018702,-2.5685244,-2.6503806,-2.748745,-2.3245125,-2.4838095,-2.6631436,-2.7600534,-2.3339076,-1.9853723,-1.9117775,-2.502066,-2.31298,-2.6479552,-2.6117263,-2.3225362,-1.952772,-1.8799783,-2.313974,-2.758351,-2.1818354,-2.4447973,-2.328857,-2.1325867,-2.4416597,-2.0614736,-2.1345778,-2.4168766,-3.0431798,-2.5518851,-2.66982,-1.4134803,-2.136697,-1.7841828,-2.2676065,-2.297422,-2.0006897,-1.7894174,-2.00877,-2.382369,-1.9310005,-1.7681879,-2.2084446,-2.7527373,-2.3055148,-2.3133376,-2.4789484,-2.1015291,-2.3257122,-2.8496892,-2.793609,-2.5795534,-3.058848,-2.7036924,-2.1575308,-2.163107,-2.6673293,-2.214351,-2.5543005,-2.701468,-2.3235826,-2.6125808,-2.3827298,-2.2475457,-2.1480403,-2.7349832,-2.763846,-2.6885455,-2.2209594,-2.3197217,-2.4817126,-2.5502844,-2.3650448,-2.3678098,-1.2476985,-1.8065611,-2.303644,-1.9453645,-2.287845,-2.403691,-1.9401081,-1.9755149,-2.469367,-2.2771158,-2.1285872,-2.200154,-2.481871,-2.4935956,-2.4382575,-2.2518187,-2.360494,-2.2790813,-2.637345,-2.4932842,-2.8569434,-2.6845496,-2.6395802,-2.1512704,-2.0696871,-2.1000087,-2.055825,-2.4474895,-2.5576253,-2.0093262,-2.42581,-2.3694596,-1.9436063,-2.3702211,-2.5513477,-2.6088183,-2.8911173,-2.4796193,-2.5386953,-2.6652215,-2.4516828,-2.6504157,-2.1926694,-1.1624203,-1.7524117,-1.9665072,-2.098941,-2.1394854,-1.3653777,-1.591715,-2.4045498,-2.4599805,-2.6343713,-2.569084,-2.0593581,-2.6269116,-2.6927273,-3.1561222,-2.5535722,-2.404902,-2.089342,-2.3793297,-2.5115945,-2.5035136,-2.5803263,-2.606177,-2.5292048,-2.3149579,-2.4570794,-2.453586,-2.100691,-2.5315778,-2.3647666,-2.2673059,-2.5415761,-2.3298965,-2.1261325,-2.2187104,-2.8484797,-2.7974246,-2.2070875,-2.5281408,-2.319637,-2.4941666,-2.6498938,-2.2760415,-1.7087415,-1.645515,-1.8195075,-1.9604659,-2.154673,-1.732033,-1.3628908,-2.1397958,-2.0732105,-2.4364321,-2.4701638,-2.2079713,-2.3886895,-2.4407868,-2.618122,-2.346194,-2.2028983,-3.0095894,-2.5492947,-2.3190386,-2.5670583,-2.3593614,-2.054419,-2.267999,-1.8807987,-2.6672995,-2.4727247,-2.0711439,-2.8006735,-2.3634315,-2.2277331,-2.6018808,-2.4449396,-2.030179,-2.5700283,-3.137783,-2.4858134,-2.3841565,-2.5274663,-2.317773,-2.1502702,-2.3869033,-1.7140626,-2.0339773,-1.7577237,-1.9706372,-1.9913206,-2.0293996,-1.8164071,-1.5947504,-2.100699,-2.3276343,-2.3558044,-1.9786943,-1.9664717,-2.3161855,-2.215294,-2.4923217,-2.3164182,-2.497064,-2.8660839,-2.5524662,-1.9555484,-1.8739142,-1.6695395,-1.9780903,-1.7540135,-1.9382432,-2.2781649,-2.1490161,-2.0155685,-2.4292624,-1.8650101,-1.8387814,-2.155203,-2.07355,-1.8182758,-2.203619,-2.1051555,-1.7882123,-2.1460075,-1.5576426,-2.1851208,-2.2907634,-2.0499973,-2.0150142,-1.3752925,-1.6006678,-2.3198242,-2.3134131,-1.7883494,-1.7579403,-1.6112334,-2.007921,-1.9214201,-2.5303924,-2.3018963,-2.207085,-2.5533433,-2.4189837,-1.9986345,-2.4657164,-2.780292,-2.6569774,-2.3250206,-2.4830387,-2.307361,-2.209745,-1.8996023,-1.8457183,-2.2650294,-2.7531312,-2.2835093,-2.0564594,-2.0829768,-1.9548936,-2.4216247,-2.223252,-2.6030915,-2.2064312,-2.130654,-2.1847916,-2.3046458,-2.177107,-1.9747707,-2.1889584,-2.2221918,-2.2180436,-2.3523343,-1.4226868,-2.093551,-2.3125567,-2.087692,-1.8889883,-1.8504151,-1.7949823,-2.2728822,-2.1208022,-2.6517105,-2.3414736,-2.3464365,-2.3398824,-2.7827199,-2.2816255,-2.4788783,-2.6873314,-2.3849778,-2.340403,-2.7121356,-2.7478209,-2.48454,-2.2539394,-2.522551,-2.558601,-2.2921398,-2.1168675,-2.2075014,-1.8936867,-2.5100682,-2.1097596,-2.4023511,-2.5520704,-2.132854,-2.164415,-2.25815,-2.3080235,-2.0168645,-2.4592657,-2.545761,-2.3879545,-2.3674188,-2.6446965,-1.5076538,-1.6592643,-2.0051508,-1.999409,-1.9736296,-1.9215565,-1.4927077,-2.4572148,-2.1043148,-2.0792887,-2.4818597,-2.367913,-2.3754501,-2.3614311,-2.659775,-2.507311,-2.539871,-2.4247448,-2.7511022,-2.5049508,-2.3952618,-2.5552347,-2.4547653,-2.4566984,-2.4242823,-2.3180149,-2.5467405,-2.329229,-2.621488,-2.69901,-2.5656352,-2.6877341,-2.4966865,-2.3984299,-2.0579836,-2.3590298,-2.3655503,-2.1528318,-2.20704,-2.0711436,-2.5398624,-2.4317644,-2.8029258,-1.5203735,-1.6883823,-1.6941984,-1.9245781,-2.0143046,-1.7445256,-2.1834612,-2.1475863,-2.4061277,-2.263443,-2.4091005,-2.394401,-2.3871384,-2.2589235,-2.5388744,-2.8627114,-2.611852,-2.5141835,-2.3676162,-2.269817,-2.4500208,-2.3393772,-2.349524,-2.8857682,-2.1634002,-2.5839415,-2.3093905,-2.6479468,-2.4804175,-2.4914827,-2.5860674,-2.7509215,-2.8754919,-2.3532617,-2.1506708,-2.7190175,-2.3978899,-2.6217518,-1.9943439,-2.0445325,-2.2807083,-2.217991,-3.1181889,-1.6025778,-1.5575337,-1.851567,-1.9562342,-1.8459824,-1.4506106,-1.7190894,-2.1837132,-2.5017266,-2.9975138,-2.4962144,-2.755111,-2.6513503,-2.4126658,-2.5797389,-2.6430876,-2.3889012,-2.4810927,-2.6363132,-2.59468,-2.6373518,-2.570338,-2.585733,-2.7506964,-2.23758,-2.633199,-1.9816874,-2.6337476,-2.636789,-2.300024,-2.2581043,-2.5680132,-2.7259512,-2.7762766,-2.6845381,-2.356053,-2.8223352,-2.2285857,-2.0678043,-1.9990638,-2.4778786,-2.5234182,-2.8699532,-1.6211286,-1.6197877,-1.9529192,-1.859198,-1.7840779,-1.5639317,-1.6911829,-2.6023912,-2.192141,-2.5194654,-2.718043,-2.8121533,-2.6482112,-2.3271055,-2.5814455,-2.7988267,-2.6596215,-2.4540536,-2.679519,-2.6379538,-2.523939,-2.6418772,-2.420299,-2.5203104,-2.7502263,-2.4520104,-2.334674,-2.4865434,-2.7457538,-2.21555,-2.6693082,-2.9951491,-2.4724388,-2.4444447,-2.4070246,-2.4228497,-2.6360064,-2.2326655,-1.9688762,-1.9346839,-2.9768124,-2.8557985,-2.6960118,-1.8050507,-1.3983252,-1.8915356,-1.8024269,-1.9190853,-1.966189,-2.0574284,-2.5628064,-2.3038838,-2.5747938,-2.9020367,-2.654533,-2.7251163,-2.4812963,-2.7911935,-2.696169,-2.72466,-2.7783844,-2.4525557,-2.7944865,-2.3395634,-2.6621675,-2.518672,-2.5395,-2.4147608,-2.467896,-2.110726,-2.1629379,-2.644536,-2.6915154,-2.8980668,-2.3519397,-2.7465053,-2.2801127,-2.4103153,-2.775875,-2.5107913,-2.2559555,-2.0909312,-1.7603275,-2.2087688,-2.142593,-2.5644689,-1.5855654,-1.5063195,-2.2837389,-2.4366562,-2.307766,-2.2639754,-2.0061333,-2.040374,-1.9753332,-2.214713,-2.4567294,-2.9094787,-2.2109253,-2.713816,-2.7105553,-2.9112294,-2.3223252,-2.5706868,-2.1151772,-2.8014925,-2.511105,-3.0936525,-2.3587718,-2.463077,-2.3703394,-2.0960307,-2.0529003,-2.224966,-2.3457541,-2.68038,-2.513341,-2.5662246,-2.8092468,-2.633752,-2.3517826,-2.3523316,-2.4903438,-2.158506,-2.4160106,-2.260521,-2.1611779,-2.2960017,-2.8567636,-1.5632031,-1.3683611,-1.9724994,-2.128353,-2.7835987,-2.4675002,-2.0252192,-2.5023615,-2.067087,-2.4103065,-2.3779206,-2.7049618,-2.9306183,-3.0144913,-2.4988463,-2.6797252,-2.2644572,-2.212984,-2.2358034,-2.326973,-2.6998587,-2.5910337,-2.445981,-2.334837,-1.8653646,-2.0513666,-1.9238117,-2.660708,-2.2059622,-2.6778395,-2.6552835,-2.5776532,-2.7129564,-2.099482,-2.4507856,-2.1757784,-2.0259516,-2.4560075,-2.3782122,-2.1043906,-2.1648872,-2.951316,-2.190056,-1.9634038,-1.6973906,-1.9226346,-2.2632015,-2.2038116,-1.8442895,-2.5542147,-2.2266371,-2.520722,-2.3170202,-2.6709502,-2.5004194,-2.4516046,-2.7688875,-2.4111938,-2.6463149,-2.5100014,-2.358199,-2.6012988,-2.5496943,-2.496814,-2.7204862,-2.4980044,-2.2775111,-2.1704934,-2.6723735,-2.2739468,-2.1306999,-2.325217,-2.118181,-2.5010622,-2.4052083,-2.3977377,-2.2904794,-2.5492556,-2.572734,-2.6953287,-2.3944142,-2.513693,-2.364567,-2.3784916,-2.557153,-2.2821672,-1.7616524,-1.8647183,-1.6361324,-2.246869,-1.5846653,-1.8675219,-2.25206,-2.0691295,-2.2587247,-2.4003677,-2.2072768,-2.16762,-2.345303,-2.3256147,-2.9674082,-2.6410418,-2.1609812,-2.7891726,-2.6460137,-2.3722742,-2.6017635,-2.365103,-2.4520028,-2.444076,-2.7985775,-2.7275367,-2.1716073,-2.5817554,-1.9275541,-1.9612416,-2.2045388,-2.5112524,-2.2299416,-2.2546766,-2.5138772,-2.76654,-2.5307724,-1.9376067,-2.1030548,-2.4881997,-2.453645,-2.6755345,-2.1513891,-1.4634724,-1.8005922,-2.0305662,-2.2050195,-2.1511295,-2.3917286,-1.9488583,-2.1271741,-2.4411001,-2.3764071,-2.3952816,-2.2181675,-2.4421575,-2.8266923,-2.300451,-2.7748377,-2.4403358,-2.7903073,-2.8759575,-2.5788665,-2.893932,-2.5558076,-2.6317177,-2.5663052,-2.4623516,-2.0402477,-2.205062,-2.7223868,-2.509494,-2.5569813,-2.7542164,-2.7999766,-2.1435192,-2.4179137,-2.5198705,-2.3427083,-2.7020042,-2.6332653,-2.1423314,-2.5466974,-2.2631905,-2.4119718,-2.3627653,-1.7285058,-1.4183822,-2.4392374,-1.9218405,-1.9491161,-1.9784101,-2.0917056,-2.3167684,-2.327633,-2.3187265,-2.550762,-2.6019037,-2.04715,-2.6313,-2.0613384,-2.331647,-2.8094938,-2.372917,-2.333927,-2.8050988,-2.7887778,-2.9691367,-2.3429134,-2.2389295,-2.2908366,-2.1185815,-2.5669065,-2.4900908,-2.4134483,-3.3035257,-2.5733304,-2.1180696,-2.2677088,-2.388009,-2.672722,-2.4270532,-2.56873,-2.6038759,-2.6875832,-2.4089239,-2.8046608,-2.85311,-2.92184,-1.8250452,-1.3977494,-2.0344877,-1.8100907,-1.6272765,-1.6967443,-1.7518932,-1.939976,-2.1253934,-2.3626819,-1.9469808,-2.443223,-2.506806,-2.7505887,-2.5519643,-2.663836,-2.5953522,-2.3919928,-2.8095298,-2.6156278,-3.1211872,-2.6689706,-2.4282727,-2.3274944,-2.749264,-2.8544462,-2.3994555,-2.248239,-2.4816973,-2.3045747,-2.6370604,-2.1074796,-2.2293954,-2.1929054,-2.7704594,-2.7462585,-2.4132514,-2.6452098,-2.552548,-2.6138558,-2.5010471,-2.1463654,-2.9330804,-1.6620919,-1.5040457,-1.8746909,-2.7517965,-2.0991147,-2.1990273,-1.8427444,-2.2252789,-2.3139608,-1.9833978,-2.2269006,-2.2910523,-2.4554496,-2.5378478,-2.5358064,-2.667449,-2.1927395,-2.6615164,-2.698733,-2.5843542,-2.7440581,-2.1582563,-2.4814708,-2.8077102,-2.8892052,-2.6439564,-2.4343455,-2.7759411,-2.328318,-2.1660545,-2.7158697,-2.5937133,-2.477111,-2.0822952,-2.7665396,-2.616504,-2.7249274,-2.5787175,-2.5338821,-3.1313157,-2.6317585,-1.9951128,-2.8049831,-1.6736642,-1.6422198,-2.2358098,-2.7016604,-1.8489034,-2.0270417,-1.4741315,-2.6271713,-2.4313738,-1.9575763,-2.1412416,-2.1764731,-1.9699601,-2.4249234,-2.5443637,-2.3704002,-1.8844206,-2.5825064,-2.3987873,-2.4845977,-2.5350797,-2.7365766,-2.2798789,-2.6297846,-3.3370996,-2.4204893,-2.2212393,-2.5120134,-2.4246964,-2.5161223,-2.1777787,-2.599886,-2.430895,-2.4630995,-2.3470328,-2.2728982,-2.8637204,-2.334847,-2.23472,-2.8738866,-2.6269684,-2.4716012,-2.776933,-1.5562625,-1.7003601,-2.48696,-2.165735,-1.9480071,-1.7039456,-1.4005835,-2.8346977,-2.2500405,-2.330419,-2.346801,-1.8064748,-1.9404484,-2.4323504,-2.4615784,-2.3913784,-2.381541,-2.4143772,-2.5036798,-2.4350166,-2.63916,-2.7572882,-2.823296,-2.9087443,-2.6472425,-2.530922,-2.6886208,-2.3913043,-2.4632657,-3.1796224,-2.4150538,-2.5905755,-3.6628878,-2.6928942,-2.2867901,-2.6231642,-2.6873283,-2.808299,-2.8612425,-2.6614034,-2.411276,-2.8136787,-2.988229,-2.4394028,-1.5140941,-2.237643,-2.01607,-2.1671495,-1.7509044,-1.7985811,-2.3336463,-2.0278819,-2.441712,-2.566139,-2.0657172,-2.8218114,-3.1681194,-2.7473223,-2.5526643,-2.5653489,-2.7042098,-2.7669296,-3.282053,-2.5788536,-2.3748481,-2.6595826,-2.4936292,-2.2151582,-2.8354073,-2.8348336,-2.3342347,-2.3035655,-2.3401756,-2.4093187,-2.5562317,-3.0584037,-2.7491338,-2.4688015,-2.8703158,-2.5171647,-2.7374861,-2.8971932,-2.8566394,-2.6531746,-2.5290475,-2.6605725,-2.3454514,-1.5470642,-2.6807485,-2.1498568,-1.8411796,-1.8373581,-1.885488,-2.3271945,-1.683234,-2.3027253,-2.191041,-2.1857464,-2.6079512,-2.765989,-2.354652,-2.3712068,-2.6608362,-2.4348948,-2.796779,-2.9592083,-2.8005407,-2.3357172,-2.4555445,-2.4759393,-2.362687,-2.6130192,-2.429399,-2.6152318,-2.5567908,-2.3604004,-2.3283885,-2.7535858,-2.806891,-2.5311875,-2.4370997,-2.8771548,-2.8769672,-2.6966367,-2.192075,-2.353093,-2.4538045,-2.7695894,-2.4884958,-1.9862944,-1.4128841,-2.1004329,-1.8788062,-1.9616387,-2.4736483,-2.136346,-2.1297512,-1.8639606,-2.460405,-1.9040146,-2.2694657,-2.6777358,-2.6695428,-2.6337132,-2.835661,-2.1504924,-2.5584548,-3.1400275,-2.6780581,-2.5553646,-2.3717034,-2.4419422,-2.292819,-2.7056634,-2.2965276,-2.584665,-2.6429396,-2.895454,-2.3812864,-2.7165186,-2.785928,-2.953409,-2.2567122,-2.7460387,-2.769287,-2.7078073,-2.4235282,-2.2928257,-2.7748568,-2.384631,-2.804413,-2.9375417,-1.8892754,-1.5384809,-1.7169493,-1.8338772,-1.7531772,-2.1812289,-1.7490978,-2.02654,-2.200205,-2.1650393,-2.249915,-2.7969975,-2.5420878,-2.7725334,-2.3482966,-2.2109265,-2.350264,-2.412151,-3.1885924,-2.5117974,-2.7995298,-2.765297,-2.2747169,-2.8468957,-3.0578072,-2.6870446,-2.2756858,-2.7822986,-2.9973392,-2.0325902,-2.4959426,-2.9863238,-3.1471946,-2.702482,-2.9376943,-2.6991732,-2.5015082,-2.3991904,-2.500572,-2.338854,-2.660575,-2.748568,-3.3814445,-2.1521208,-1.283694,-1.9489022,-2.2978349,-1.9494824,-2.257917,-2.0691698,-2.1913273,-2.4932806,-2.12604,-2.2822807,-2.548719,-2.9040165,-2.7821944,-2.7376773,-2.5528362,-2.6932216,-2.798,-2.5258043,-2.0696025,-2.5751164,-2.3728886,-2.4340913,-2.7254317,-2.5308194,-2.2990682,-2.261872,-2.777723,-2.494072,-2.339108,-2.4418035,-2.861606,-2.7576747,-2.7980368,-2.9173903,-2.913653,-3.094271,-2.8237822,-2.7019513,-2.1806731,-2.4295254,-2.862601,-2.6140075,-2.3745258,-1.7610335,-2.1104834,-2.4731803,-2.1406343,-1.7323828,-2.2663648,-2.3677104,-2.4810758,-2.3672805,-2.453672,-2.4761171,-2.7203584,-2.317949,-2.5863295,-2.5135102,-3.2536116,-2.6058118,-2.4950843,-2.8802445,-2.6972585,-2.804211,-2.6573613,-2.978581,-2.4163537,-2.3235102,-2.2685542,-2.8178957,-2.493475,-2.4446864,-2.33054,-2.746891,-2.7695217,-2.676197,-2.856253,-2.4135022,-2.6001074,-3.2187,-2.3450968,-2.4303153,-2.709948,-2.6600482,-3.1754403,-1.9611897,-1.765146,-2.3148997,-2.2790585,-2.2534232,-1.9452212,-1.9684591,-2.3970327,-2.6190624,-2.3374586,-2.705283,-2.646646,-2.7987983,-2.16429,-2.3257403,-2.2768214,-2.7555926,-2.7402346,-2.7043977,-2.780281,-2.4665277,-2.8037858,-3.1813252,-2.5678194,-2.5917404,-2.2917607,-2.8500197,-2.7672997,-2.9003632,-2.5253766,-2.2188182,-2.4249506,-3.0654564,-2.410219,-2.8616056,-2.5961387,-2.560361,-2.7447739,-2.4592478,-2.7197187,-2.991936,-2.826738,-2.8778946,-1.657362,-1.578177,-2.0869071,-2.3175373,-2.4484525,-2.1337242,-1.9018527,-2.3269658,-2.6862173,-2.9027913,-2.5406506,-2.840916,-2.859023,-2.633157,-2.499038,-2.464246,-2.5626905,-2.6946156,-2.7073467,-2.6636178,-2.5012271,-2.712262,-2.9233384,-2.6455777,-2.794414,-2.1431012,-2.5785892,-2.4597526,-2.9389734,-2.5575242,-2.9194238,-2.722062,-2.7984397,-2.3437304,-3.0569584,-3.0707493,-2.656842,-3.0043294,-2.6419537,-2.5492623,-2.6402383,-2.7899382,-2.5466118,-1.5439646,-1.3157384,-2.4076695,-2.3338492,-1.9228113,-1.9593967,-1.9605861,-2.303486,-2.4714062,-2.6605165,-2.4522762,-2.7572813,-2.907097,-2.2108572,-2.6465247,-2.7843976,-2.0893393,-3.025389,-2.4996073,-2.8674102,-2.6547558,-3.2807357,-3.1366482,-2.797872,-2.125952,-2.3285267,-2.5203605,-2.858613,-2.796814,-2.8703325,-2.5536337,-2.8494473,-2.2635834,-2.335959,-2.5457067,-2.5747821,-2.6783226,-2.3192425,-2.9940927,-2.7483826,-2.5890834,-2.8193307,-2.519167,-1.5337203,-1.4959066,-2.2145052,-2.0993311,-1.7336973,-1.4768643,-1.8084419,-2.4895363,-2.2542975,-2.464471,-2.333111,-2.9254358,-2.2818599,-2.438856,-2.2431202,-2.570235,-2.1050382,-2.886641,-2.6281776,-2.5585163,-3.1900873,-2.8037665,-2.7383587,-2.169659,-1.867725,-2.395132,-2.6103492,-2.802909,-2.917902,-2.8924265,-2.568543,-2.7615628,-2.4993296,-2.4723794,-2.7659154,-2.5103853,-2.5144696,-2.3261757,-2.6881316,-2.7047236,-2.6558232,-3.1036603,-3.1033962,-1.856896,-1.6953926,-2.438071,-2.5898292,-1.95704,-1.7223961,-1.8918524,-2.368554,-2.2971451,-2.3165863,-2.3259304,-2.620351,-2.5110874,-2.3624208,-2.5103023,-2.5547812,-2.484113,-2.4753702,-3.1437833,-2.1857615,-2.9621122,-2.484526,-2.4874105,-2.3816295,-2.379218,-2.5023546,-2.765995,-2.9711268,-2.7987754,-2.337428,-2.4840107,-2.5134845,-2.2092419,-2.6118412,-2.7381985,-2.424842,-2.3615477,-2.9139113,-2.8809426,-2.4061935,-2.330431,-2.779798,-2.6280403,-1.9874432,-1.7393104,-2.1826043,-2.1501653,-2.0218408,-2.1535764,-2.391291,-1.9852273,-2.242775,-2.5464182,-2.630387,-2.321213,-2.7515118,-2.5637748,-2.436708,-2.4849494,-2.4679916,-2.632135,-2.8578591,-2.490207,-2.6255429,-2.3186677,-2.5002,-2.3429005,-2.4210002,-2.392046,-2.4408605,-2.6891773,-2.3176904,-2.4545827,-2.213203,-2.6638882,-2.51602,-2.560154,-2.9160404,-2.6533942,-2.7010553,-2.7830982,-2.6627872,-3.0079806,-2.5992856,-2.5237303,-2.5383701,-1.9453135,-1.8648015,-1.9742082,-1.9799184,-2.4240406,-2.079187,-2.1956708,-2.1560462,-1.8486509,-2.4562473,-1.8733865,-2.1343172,-2.4686937,-2.5502877,-2.4813437,-2.9217186,-2.3415666,-2.6080632,-2.7008061,-2.6084673,-2.3916328,-2.395848,-2.25764,-2.7235334,-2.1707563,-2.3944297,-1.8981419,-2.447577,-2.1862392,-2.437807,-2.5666049,-2.6461537,-1.7692387,-1.8751699,-2.647725,-2.6045494,-2.5804193,-2.7921152,-2.5764441,-1.8561735,-2.5327718,-2.2613838,-2.4754407,-1.9640017,-2.1873045,-1.7938854,-1.6726366,-2.4599984,-2.5653672,-2.260148,-2.1307974,-1.7157873,-2.6659415,-2.0352895,-2.202602,-2.7352185,-2.4661741,-2.3567023,-2.8821194,-2.5399318,-2.6021159,-2.6254652,-2.7063148,-2.6077602,-2.8683298,-2.082297,-2.5903857,-2.594614,-2.3080726,-2.3712285,-2.311795,-2.4155989,-2.5804465,-2.90438,-2.5512793,-1.9217693,-1.8458183,-2.7124493,-2.634523,-2.62047,-2.4332697,-2.6456656,-1.8042529,-2.7214205,-2.8398023,-2.3454738,-1.6611897,-1.7871627,-2.284102,-1.9845283,-1.9767878,-1.8225011,-2.2859397,-1.8736748,-1.9072077,-2.908669,-2.7200017,-2.5988288,-2.6393602,-2.2857378,-2.512804,-2.8236797,-2.4909468,-2.5186744,-2.4455714,-2.5334718,-2.7034657,-2.467397,-2.2556074,-2.3744168,-2.3098826,-2.121579,-2.2667358,-2.4222057,-2.7565722,-2.0231855,-2.4967158,-2.878872,-2.5066736,-2.5124407,-2.7005558,-2.8328078,-2.8051403,-2.3508956,-2.6473422,-2.769179,-2.6446614,-2.3142045,-2.569591,-2.0266037,-1.6474288,-2.2053645,-2.3351288,-1.7251276,-1.863536,-2.4186172,-2.2817338,-2.0217738,-2.4620602,-2.67815,-2.9227817,-2.219708,-2.2589345,-2.1453233,-2.831767,-2.1639256,-2.5379272,-2.8405359,-2.9217756,-2.4561214,-2.4682019,-2.4391263,-2.389312,-2.2367537,-2.3928711,-2.095531,-2.437504,-2.416055,-2.2448063,-2.3164985,-2.1668317,-2.4539218,-2.8172498,-2.9022193,-3.1152742,-2.528457,-2.8538067,-2.4314315,-2.4046416,-2.4937568,-2.6144733,-2.9764078,-1.9928249,-1.5937752,-1.9909472,-2.3731365,-1.795911,-2.0332265,-2.6491497,-2.2763853,-1.7758015,-2.1291587,-2.8227727,-2.270486,-2.6540134,-1.9976848,-2.3777819,-2.764395,-2.069938,-2.8574624,-3.429665,-2.9051683,-2.6290488,-2.2102513,-2.8106773,-2.3377302,-2.1182773,-2.5612884,-2.4236715,-2.5134287,-2.4326532,-2.6222436,-2.311521,-2.2124476,-2.1888773,-2.715026,-2.8914661,-2.5928178,-2.5616202,-2.5350997,-2.0702546,-2.4900773,-2.8140733,-2.9753735,-2.5091085,-1.7481345,-1.6063854,-2.1986802,-1.914109,-1.9011003,-1.9079323,-2.0127091,-2.2236614,-2.021334,-1.8681343,-2.5244012,-2.0123777,-2.4089622,-1.947587,-2.8795636,-2.8385086,-2.6038606,-2.71669,-2.816362,-2.2865205,-2.2304127,-2.0792167,-2.5541074,-2.4137564,-2.6192672,-2.5879445,-2.9160845,-2.3818002,-2.2847304,-2.5131724,-2.086932,-2.3562589,-2.1068285,-2.670342,-3.2194564,-2.7742937,-2.7815347,-2.7619495,-2.5093975,-2.2064922,-2.8853927,-3.0555127,-2.8660543,-1.7284148,-1.764431,-2.2679832,-2.047334,-2.0400329,-1.9236076,-1.6063633,-2.1923504,-1.9961389,-2.3594682,-2.66096,-2.1052818,-2.5069988,-2.5578988,-2.546369,-3.2212183,-2.7960951,-2.6603773,-2.9415677,-2.9152367,-2.5591025,-2.5242488,-2.3308516,-2.2607784,-2.932302,-2.616103,-2.7973423,-2.8767352,-2.5528646,-2.3575306,-2.2091384,-2.5872102,-2.189933,-2.501402,-2.7234974,-2.3993912,-2.4347088,-2.5403678,-2.804044,-2.5897913,-2.5092292,-2.7170613,-2.698148,-0.1583979,0.5687513,-0.41277125,-0.8703364,-1.7425964,-1.5208187,-2.372228,-0.8609016,-0.03814471,-0.19430685,-0.3782577,-1.184945,-1.3726258,-1.3121612,-1.7090749,-1.3931952,-0.7682466,-0.7321356,-1.3127513,-2.4012659,-2.469317,-2.2657547,-2.6679027,-2.720457,-2.7076905,-2.222641,-2.8118217,-2.588639,-2.18563,-2.2121494,-2.3631935,-2.747664,-2.4828453,-2.6709523,-2.8395674,-2.5989802,-3.0861752,-2.8002038,-2.6982007,-2.618849,-2.717914,-2.4221523,-3.1346774,0.57930255,1.0791861,0.32213005,0.88036215,-0.40102783,-0.035476565,-0.56035435,0.41094682,1.0782559,0.6217911,0.54393274,0.3482077,-0.10025169,-0.046871074,-1.1825917,-0.3647044,-0.012120584,0.360564,-0.25742432,-1.4379382,-1.69671,-2.2577462,-2.4079247,-2.46349,-3.0197883,-2.3716497,-2.9371347,-2.5293958,-2.4005208,-2.8260071,-2.7554636,-2.6734579,-2.7699645,-2.9668741,-2.8099988,-2.4958394,-2.5201437,-2.8926358,-2.9225533,-2.412983,-2.608121,-2.6596215,-3.038219,0.6912034,1.0428907,0.96746284,1.2372395,0.23405865,0.4584207,-0.4320474,0.7924134,1.4701618,1.2730261,0.9289484,0.7323232,0.25702322,0.40282816,-0.30632615,-0.12276809,0.36887726,0.51495165,0.13992791,-0.783658,-1.6073982,-2.3020918,-2.0325978,-2.5179331,-2.743584,-2.327819,-2.5148377,-2.5715818,-2.2913516,-3.6402562,-3.0730546,-2.2956586,-2.1894248,-1.9993236,-2.4906394,-1.7823331,-2.080744,-1.9565011,-2.305422,-2.3837595,-2.2448745,-2.7617078,-2.9419456,0.4559302,0.7146779,1.4405899,1.6967033,0.5745591,0.75021917,0.48694572,0.46347597,1.6320921,1.5033966,0.9495282,0.93888557,0.3159882,0.63399243,0.3278415,0.27732965,0.72672886,0.8449425,0.7532941,-0.4657857,-0.9994301,-1.4278685,-1.7731801,-1.8230455,-2.0461168,-2.4282286,-2.8428428,-2.2058074,-2.2003345,-2.6272662,-2.5612001,-2.2735412,-2.0778928,-1.1591794,-1.419154,-1.2535919,-1.5801971,-2.235655,-2.1050537,-2.0801637,-2.642097,-2.7708306,-2.7414956,0.111580566,-0.15133044,1.5157998,2.1459367,0.72829866,0.76785654,0.49449825,0.8160381,1.7856246,0.87959284,0.7613456,0.86398554,0.4922868,0.77311313,0.0905076,0.20614369,0.37379122,0.55152375,0.8286945,-0.13788283,-0.48647115,-1.3183578,-1.6136276,-1.7170317,-1.7882456,-2.3429818,-2.8412848,-1.3755413,-2.344886,-2.4495735,-2.3325036,-2.1265504,-1.7936679,-0.848633,-0.66069406,-0.45715958,-1.5367322,-1.7402837,-1.8397249,-2.3640523,-2.270209,-2.2299488,-2.4676213,0.009969509,0.24762152,1.0470027,2.4623723,0.49810132,0.8055407,0.13022228,1.0906264,1.4878091,1.1612921,0.73584837,0.671565,0.53606004,0.014399374,0.53951776,0.49822626,0.5152779,0.98377734,0.9290898,0.07065399,-0.8908118,-1.352299,-1.7331481,-1.454405,-1.9605772,-2.343985,-2.7726884,-1.38898,-1.748713,-2.4401658,-2.8047905,-2.2300146,-1.9526823,-0.734874,-0.5062368,-0.44163463,-1.2836068,-1.8813708,-1.6609026,-2.0570855,-2.3273466,-1.9487075,-2.7377486,0.20612471,0.9145072,1.1746944,2.6570342,0.5237222,0.58875835,0.5399735,0.87525356,1.0089839,1.6859246,1.1202626,0.5755814,0.20872262,0.22737284,0.91683596,0.44378382,0.76220745,0.9391858,0.9547014,0.05198006,-1.0396742,-1.8554486,-1.9725183,-1.546988,-2.2778022,-2.4479046,-2.2556005,-1.3944595,-1.4607472,-2.1754189,-2.451951,-2.5058048,-1.981338,-1.0765965,-0.5618466,-0.17328316,-1.377351,-1.458214,-1.284817,-1.9987226,-2.3667057,-2.332536,-2.9677427,0.14949687,0.9256163,1.589871,2.7811215,0.6196379,0.3870245,0.1442245,0.67489225,1.2435308,1.8702209,1.1494632,0.748173,0.16978346,0.9210651,0.9293152,0.45623606,0.86565405,0.69378895,0.86544514,-0.09497114,-0.7638013,-1.5818582,-2.2053306,-1.1457686,-1.753182,-1.559747,-2.1005442,-1.4444327,-1.8690591,-2.2360084,-2.237828,-2.505726,-2.2039976,-0.7438542,-0.65366566,-0.10502297,-1.2621089,-1.7907888,-1.2618798,-1.8252057,-2.3959372,-2.3638678,-2.189145,0.13644074,0.5775115,1.9774711,2.9378858,0.43408817,0.33255506,-0.072567455,0.5564125,1.109013,2.1921356,1.0144103,0.8787591,0.65342444,1.2851695,0.27134892,0.59894925,0.63955235,0.5532163,0.5544085,0.24077637,-0.47036773,-0.9175516,-1.5722182,-1.1169597,-2.1340055,-1.8679094,-1.7790282,-1.300985,-1.3176802,-2.205137,-2.3314006,-2.0484447,-2.3333263,-0.7622495,-0.6676795,-0.015382706,-1.2374276,-1.8984,-1.3804753,-1.9302092,-2.5887747,-2.1990964,-2.5436158,0.35797688,0.82653433,1.9205315,3.036205,0.3889946,0.14011621,0.16296704,0.80296516,1.1047137,2.355474,0.99377114,0.92781234,0.80605704,1.5401957,0.5877657,0.6852857,0.64153296,0.80997473,0.864862,0.4966115,-0.30801335,-0.8088454,-1.5803251,-1.3869978,-2.0512104,-1.5349377,-1.8621511,-1.38061,-1.3566543,-2.3687754,-2.3017066,-2.3147733,-2.1502512,-1.0257372,-0.7123542,-0.05167594,-1.1527846,-1.5747999,-1.4099452,-1.799813,-2.177296,-2.2744446,-3.1606088,0.52960813,0.92205006,1.8749877,3.0902014,0.48608378,-0.16062644,0.1477522,0.8793797,0.8944823,2.4545543,0.94243944,1.033389,0.72025245,1.5588833,0.5927949,0.54038435,0.76432997,1.0023992,1.0268489,0.68876934,-0.48179448,-0.9357202,-1.7945794,-1.1595458,-1.7504542,-1.7259272,-2.0727584,-1.3502289,-1.500478,-2.2338748,-2.35972,-2.3788633,-1.6644658,-1.1693367,-0.9506529,-0.575818,-1.3358473,-1.8497821,-1.6416452,-1.8372345,-2.529017,-2.259102,-2.645498,0.4151417,1.0138123,2.241836,3.0980978,0.3396527,0.36983988,-0.051683847,0.6864048,1.7688607,2.4837108,1.1933897,1.2051338,0.81873506,1.4357446,0.954027,0.6884619,0.6165787,1.0171255,1.0178425,0.5099726,-0.6752229,-1.2270488,-1.7499881,-1.4610292,-1.4288903,-2.4746556,-2.1095603,-1.0630573,-1.8365871,-2.2197783,-2.4581501,-2.5668073,-1.776462,-1.7379827,-1.0704985,-0.5441243,-1.201953,-2.2607865,-2.0522053,-1.9536988,-2.75931,-2.2759173,-2.957107,0.53305185,0.97515583,2.5180953,2.8005548,0.45655194,0.7785913,0.37050465,0.5669292,1.8928409,2.0804498,1.4433045,1.3264083,0.7358873,1.4521701,0.9367826,0.92150646,0.46294978,1.1055137,1.1691877,0.49445367,-0.93503875,-1.1849116,-1.5064185,-1.9662744,-1.9839152,-2.0539556,-1.9301934,-1.137224,-1.5425777,-2.2770724,-2.3579886,-2.547006,-2.0563111,-1.1772695,-1.085541,-0.33084133,-0.8657512,-2.240284,-1.6022589,-2.3332953,-2.624803,-2.426539,-2.6528332,0.7626232,1.1778291,2.600415,2.4873145,0.22787511,0.7631583,0.7585202,0.6139722,1.9502575,1.6256485,1.3993089,1.1761467,0.33434755,1.5557474,1.0362178,1.0102485,0.7190331,0.9476909,0.9626823,0.4774646,-0.69959,-1.0718988,-1.6509783,-2.067154,-2.018773,-2.297862,-1.8836906,-1.2156544,-1.6815531,-2.0043314,-2.627207,-2.2712069,-1.7947297,-1.8109487,-1.2567018,-0.4534392,-0.9017819,-1.5604656,-1.3123466,-2.0278833,-2.8031404,-2.5803387,-2.7310736,0.6812604,1.3175033,2.679468,2.7537181,0.4060451,0.8501717,0.79532826,0.80359197,1.9488298,1.8064982,1.3245493,1.1492577,0.7144895,1.4968182,1.2110662,0.8343108,0.7863116,0.6828238,0.67607987,0.15323363,-0.91577566,-1.8002499,-1.994777,-1.9258279,-2.7274425,-2.3153923,-2.506164,-1.2853935,-1.6399968,-2.0118454,-2.6640081,-2.1180532,-1.9264774,-1.7913747,-1.2254138,-0.45320114,-0.8471744,-1.9760655,-1.225515,-2.1201303,-2.6072311,-2.7349977,-2.7315762,0.6736571,1.3320961,2.7024934,2.6880083,0.38237166,1.1223522,0.94631547,1.2613908,1.9211628,2.0028534,1.5920601,1.331803,0.8760071,1.6977799,0.55163956,0.8195935,0.5256385,0.91410536,0.97320086,0.4748067,-0.59981036,-1.7149897,-2.2347114,-2.2665634,-2.0366158,-2.514921,-1.8742658,-1.6826855,-2.1188204,-2.5839682,-2.6453736,-2.1931572,-2.2720485,-1.7978944,-1.6527705,-0.81851023,-1.1662674,-1.9109173,-1.8548728,-2.3994155,-2.7608602,-2.7648926,-2.5885417,0.83452165,1.3487358,2.5678616,2.4544306,0.6592043,1.1835084,0.99033564,1.4077731,1.9681332,2.1826024,1.5520291,1.344192,1.1223485,1.6045328,0.99137074,0.73057294,0.7829534,0.92152774,0.92154825,0.40958354,-0.53714174,-1.0849447,-1.8687645,-1.9061302,-1.7032554,-1.9975929,-1.6751409,-1.0270696,-1.4240073,-2.5817175,-2.4431365,-2.0732248,-2.030425,-1.4910436,-0.98469555,-0.40464818,-0.9300957,-1.7786068,-1.6384128,-2.120342,-2.9170594,-2.6089542,-3.1799774,0.7704655,1.3559262,2.4438753,2.327303,0.398071,1.1239213,0.919086,1.3445768,2.2120202,1.8466018,1.4746462,0.9848106,0.9616488,1.5275234,1.1439158,0.5121637,0.2957761,0.80128086,0.6648076,0.52743816,-0.6480705,-1.2546502,-1.9028339,-1.9518307,-1.9730883,-2.1266482,-1.8035655,-1.2336696,-1.0899644,-2.1661458,-2.2274566,-2.6973865,-2.2737815,-1.326253,-1.0569535,-0.14122328,-0.52234495,-2.1243525,-1.2889843,-1.8501798,-2.5997834,-2.859285,-2.7984467,0.7971771,1.2791976,2.2658577,2.1428537,0.60636944,1.1659299,0.9004467,1.3510607,2.4065788,1.7689024,1.529962,0.9836003,0.6701074,1.4911666,1.0611384,0.63617665,0.61325854,0.48256415,0.8383699,0.58234614,-0.57352734,-0.98749584,-1.973269,-1.7310385,-1.7344548,-2.18257,-1.7684134,-1.0544912,-1.0640018,-2.0592573,-2.5221148,-2.4116037,-1.7410101,-1.2670001,-0.8387804,0.30484387,-0.15840563,-1.4333313,-1.1673518,-1.2172935,-2.1440513,-2.8154285,-2.2265062,0.8347461,1.2923813,2.0552874,2.1875868,0.78548235,1.1430817,0.6676591,1.3590682,2.1814175,1.809168,1.1053469,0.8660653,0.5748482,1.2560202,1.0438137,0.7546371,0.4245468,0.61712474,0.771609,0.45391968,-0.48534104,-1.0546013,-1.7592547,-1.6398951,-1.9645566,-2.2239144,-1.9074014,-1.2689477,-1.4054867,-2.2510836,-2.3460057,-2.2461069,-2.164725,-1.7457781,-1.1118374,-0.103668004,-0.82129514,-1.5305816,-1.8742633,-1.4040865,-2.2500484,-2.6388342,-2.2529168,0.71709114,1.2139704,1.9564987,2.0429254,0.8211185,1.1829922,0.10952745,1.3665872,2.0960653,1.4967084,0.85334945,0.7932637,0.33212993,1.2578635,0.6503524,0.74038684,0.55606943,0.47080278,0.7912457,0.044227418,-0.5160379,-1.190169,-2.148831,-1.467844,-2.081429,-2.3676453,-1.9863102,-1.2357124,-1.9570376,-2.6565406,-1.9051205,-2.3352098,-2.5517642,-2.2890127,-1.1013296,-0.3559965,-1.1988616,-2.0355606,-1.7395139,-2.3755577,-2.2748172,-2.5781746,-2.7137008,0.7632892,1.0751252,1.8543922,1.9824811,0.90707433,1.0733945,-0.01770156,1.3623594,1.8691444,1.5768629,0.5996574,0.68435,0.40281937,0.89946,0.96610934,0.61194664,0.3856558,-0.056789625,0.20654456,0.15789577,-0.3345851,-1.0357504,-1.9172127,-2.0298727,-2.382943,-2.4054139,-2.2285507,-1.5679942,-1.761219,-2.1365268,-1.5209557,-2.1291912,-2.0883644,-1.9658567,-1.3386714,-0.6146545,-1.0004686,-2.2075071,-1.7649888,-1.9240716,-2.450824,-2.7297127,-2.9157214,0.6942319,0.81977516,1.5412155,1.6423101,0.95226073,0.9171937,0.31128782,1.0088303,1.27877,1.4273497,0.03973026,0.24513417,0.30365407,0.7240246,0.79002917,-0.082551956,0.19898923,-0.097550176,0.13270792,0.091211274,-1.2198236,-1.4148722,-1.826706,-1.7512975,-1.8676474,-2.284549,-2.4205794,-1.8082082,-1.725962,-2.1505828,-2.227087,-2.085046,-2.0962107,-1.7437791,-1.6030365,-0.688405,-0.87128454,-1.9709244,-2.408696,-2.2428832,-2.2728631,-2.2537632,-2.3768518,0.6448595,0.87937087,1.2035477,1.4941661,0.8181104,0.79853636,-0.34635487,0.3663373,1.0770329,0.88398415,0.08808809,-0.101601,-0.4217052,0.589727,0.45411175,-0.0068312064,0.06747065,0.047699485,0.45681497,0.24447933,-1.5066067,-1.6844342,-1.7634463,-1.7058171,-2.071288,-2.214991,-2.1813493,-2.0520833,-2.155058,-2.4746304,-2.656717,-2.5863547,-2.481461,-2.7513256,-1.8217515,-0.9260481,-1.6865414,-2.2759893,-2.7997656,-2.5951662,-2.8169522,-2.4478087,-2.704612,0.7173499,0.92026967,1.035503,1.1985672,0.3603228,0.6487528,-0.17895429,0.3765699,1.3473692,0.7710323,-0.42850107,-0.30720356,-0.08308861,0.30108705,0.5670511,-0.22176936,-0.8042122,0.08890954,0.21425064,-0.08170038,-1.1165205,-1.26809,-2.2685163,-2.002726,-2.455167,-2.3787992,-3.1269238,-2.5716245,-2.4534094,-2.548161,-2.4649382,-2.5729053,-2.6783764,-2.4668992,-1.9861312,-1.1313317,-1.8320677,-2.6625972,-2.6100035,-2.3191485,-2.7069101,-2.9276989,-2.911418,0.44801408,0.82808226,1.3570895,0.6619353,-0.14661516,-0.043907836,-0.43373823,0.16705632,1.0651236,0.546747,-0.09246949,-0.87409204,-0.30665252,0.24426478,0.14257652,-0.5214105,-1.1291187,-0.5360896,-0.4254629,-0.37591404,-0.65812945,-1.4094977,-2.2519171,-1.8702028,-1.7824049,-2.0207467,-2.9148064,-2.8632727,-2.4799538,-2.8219273,-2.2622252,-2.4173722,-2.9531069,-2.3151636,-2.069083,-2.1113906,-2.183779,-2.5953872,-2.448177,-1.8391278,-2.2189782,-2.8404121,-2.576121,0.66400254,0.43481317,0.90653306,0.6178144,-0.4169601,-1.1484407,-0.9945558,-0.14862189,1.2547767,0.25184095,-0.6067953,-1.2914425,-0.9525747,-0.06609761,-0.07612386,-0.8032796,-1.1562157,-1.1373699,-0.4354943,-0.92595357,-0.52744913,-1.8661642,-1.9975681,-1.9421456,-2.1519387,-2.2246108,-2.2728906,-2.3576367,-2.4373002,-2.5080183,-2.1818445,-2.198665,-2.2690969,-2.3020933,-1.9593626,-2.2866828,-2.3659327,-2.93583,-2.58773,-2.14393,-2.6331244,-2.490356,-3.1732063,0.28706655,0.13449042,0.93997604,0.17503798,-0.38079092,-0.9888566,-1.6666515,-0.65783435,0.7992757,0.042813916,-1.2670591,-0.86892045,-1.0753851,-0.2778512,-0.31257135,-1.5350583,-1.1206862,-1.065821,-1.1210884,-1.1092637,-1.1039277,-2.376141,-2.1837928,-2.0780292,-2.1187117,-2.1826599,-2.3274252,-2.523129,-2.3368602,-2.351469,-2.0989816,-2.4665918,-1.9315178,-2.2243693,-2.2814717,-2.5898626,-2.6150503,-2.8403568,-2.4065082,-2.2686315,-2.7223861,-2.449998,-2.7085357,0.37446785,0.18497284,0.41607463,-0.4021794,-0.99659055,-1.2132268,-1.5632519,-1.2386613,-0.057807926,0.09684995,-1.5473769,-1.3918712,-1.0002899,-0.61457723,-0.32023102,-1.6351907,-1.3568555,-1.3136563,-1.6050596,-1.5103704,-2.066146,-2.7121263,-2.485112,-1.7903135,-2.045974,-2.5928257,-2.5997264,-2.1659985,-2.534585,-2.3885531,-2.3490584,-2.6881464,-2.5364602,-2.2885356,-2.696361,-2.3422496,-2.253679,-2.4632003,-2.596916,-2.1782887,-2.3944316,-2.4984767,-2.4271574,0.099038325,-0.08968939,0.42984226,-0.075616546,-1.1658663,-1.0109423,-1.415246,-0.70991445,0.26741853,-0.46385702,-1.2291104,-1.3048991,-1.3943082,-0.65706044,-0.95433456,-1.5825379,-1.4223834,-1.6610167,-1.6050018,-1.7721945,-2.0811658,-3.106897,-2.4221904,-2.2002277,-2.4938517,-2.4651744,-2.4871414,-2.0921898,-2.8099422,-2.2548642,-2.102267,-2.1731167,-2.5520053,-2.165187,-2.4935904,-2.4098475,-2.3013818,-2.4721365,-2.5220642,-2.6274571,-2.6905031,-2.4666562,-2.7366107,0.14551955,-0.5099508,0.47311887,0.15003717,-1.4566354,-1.1295279,-1.2381387,-0.5436982,-0.3887125,-0.35804608,-1.4448094,-1.2849793,-1.5568454,-1.1879199,-1.018829,-1.1961371,-1.9107686,-2.061314,-1.7322035,-2.170622,-2.4320798,-2.1738405,-2.0969462,-2.194874,-2.5379503,-2.1326869,-2.0441236,-2.7279036,-2.3347204,-2.590105,-2.531353,-2.421173,-2.1822152,-2.447149,-2.2782457,-2.4065685,-2.5688853,-3.1581876,-2.367254,-2.368078,-2.8862705,-2.3930197,-2.503763,-0.04742834,-0.09566385,0.26873648,-0.6095479,-1.3676565,-1.3066583,-1.67756,-0.98815054,-1.1526698,-0.4285526,-1.5344131,-1.4123365,-1.3703291,-1.194842,-1.821943,-1.647411,-1.876707,-1.798641,-2.0784903,-2.24333,-2.0158424,-2.2867773,-2.4699812,-2.5987415,-2.7026017,-2.198166,-2.2908018,-2.8950791,-2.1249778,-2.5918152,-2.5100543,-2.2985234,-2.164378,-2.2910578,-2.2354805,-2.2115877,-2.680328,-2.940894,-2.8476508,-2.7842393,-2.604194,-2.464326,-2.6030684,0.118021056,-0.20658219,0.1839202,-0.37086084,-1.1768672,-1.2826282,-1.632316,-1.1544268,-0.5745815,-0.7140428,-1.0781635,-1.3789848,-1.4715102,-1.1114657,-1.4476454,-1.7565953,-1.4061548,-1.3920029,-1.300747,-1.3377293,-1.8748776,-2.7453904,-2.2645788,-2.9183407,-2.2522717,-2.5872514,-2.0624156,-2.4595885,-2.4299183,-1.8374914,-1.841401,-2.3315573,-2.122216,-1.8554355,-2.8049662,-2.25957,-2.6021862,-2.4632592,-2.4112651,-2.7151449,-2.7807446,-2.040098,-2.351747,0.2899891,0.6475088,0.3897832,-0.62095946,-1.214536,-1.3120526,-1.5108597,-1.1848425,0.09021924,-0.44150037,-1.3129367,-0.88449514,-1.1861769,-1.161405,-1.0554984,-1.572293,-0.87709534,-0.931463,-0.39029044,-0.8216047,-1.3930074,-2.06451,-2.2443578,-2.3762665,-2.0495348,-2.1263115,-1.9768467,-2.007253,-2.5403552,-1.9729503,-2.0973372,-2.4853303,-2.195546,-1.8479092,-2.8254626,-2.989606,-2.3782332,-3.1313741,-2.21858,-1.9675859,-2.2005,-2.0712712,-2.1737041,0.4753845,1.0117333,0.75459665,-0.66105545,-1.412339,-1.2318835,-1.4542505,-1.3693655,0.22883658,-0.906071,-1.7748824,-1.4665644,-1.0553129,-0.9410113,-0.8553746,-1.220233,-0.89124304,-0.4060366,0.08632546,-0.49762917,-0.885161,-1.2781318,-1.9989384,-2.2581737,-1.9345112,-2.0405476,-2.1171324,-1.9461062,-2.7410705,-2.196697,-2.3306992,-2.6228197,-1.935201,-2.188272,-2.2811,-2.7390735,-1.9810408,-2.8242898,-2.5979452,-2.0167627,-2.1247652,-2.4024148,-2.5684488,0.5725974,0.7365229,0.45086163,-0.46725523,-1.4024324,-1.4138924,-2.0617673,-1.4919128,0.08512024,-1.0463885,-1.6988274,-1.336686,-1.5878621,-1.2181134,-0.9340508,-1.1932557,-1.3672342,-0.58017856,0.1431367,-0.26959062,-0.68027514,-1.0862035,-1.918091,-2.0426452,-2.0051947,-2.4494789,-2.4385507,-2.2584066,-2.4635475,-2.209678,-1.9657546,-2.4924707,-2.2840364,-1.8482293,-1.7166471,-1.8577135,-1.4793687,-2.3122878,-2.3144467,-1.7266362,-1.8610185,-2.5788422,-2.4419029,0.6059894,0.09369359,0.31690952,-0.87224907,-1.7548954,-1.3750881,-2.0365965,-1.7850358,-0.3756121,-0.9712873,-1.4751863,-1.3932837,-1.6312314,-1.3832946,-1.4497806,-1.6928535,-1.2986636,-0.8989356,-0.5908274,-0.9821057,-0.91007155,-1.66665,-2.3999255,-2.020145,-2.5018504,-2.6080053,-2.6345546,-2.566768,-2.4258058,-2.099182,-2.3339283,-2.6038556,-3.401814,-2.2612286,-1.5396799,-1.5671581,-1.9580657,-2.4340284,-2.281765,-2.3110995,-2.1305723,-2.3123076,-2.8473256]) 
s1=s1.reshape(1,98,43,1)

print(s1.shape)
predictions=model2.predict(s1,batch_size = 1)
print(predictions.shape)
print(predictions[0])

(797, 98, 43, 1)
(1, 98, 43, 1)
(1, 4)
[1.4187780e-10 9.9993598e-01 1.1634778e-06 6.2832900e-05]


In [104]:
a="1222"
b=5555
print(a+str(b))

12225555


## **模型转化**

In [149]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)

# Convert the model to the TensorFlow Lite format with quantization
def representative_dataset_gen():
    for i in range(0, len(spectrograms), 100):
        yield [np.array(spectrograms[i:i+100])]
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset_gen
model_tflite = converter.convert()

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)

35360

In [150]:
# Calculate size
size_tf = os.path.getsize(MODEL_TF)
size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)
size_tflite = os.path.getsize(MODEL_TFLITE)

# Compare size
pd.DataFrame.from_records(
    [["TensorFlow", f"{size_tf} bytes", ""],
     ["TensorFlow Lite", f"{size_no_quant_tflite} bytes ", f"(reduced by {size_tf - size_no_quant_tflite} bytes)"],
     ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"(reduced by {size_no_quant_tflite - size_tflite} bytes)"]],
     columns = ["Model", "Size", ""], index="Model")

,Size,
Model,,
TensorFlow,0 bytes,
TensorFlow Lite,127324 bytes,(reduced by -127324 bytes)
TensorFlow Lite Quantized,35360 bytes,(reduced by 91964 bytes)


In [151]:
!xxd.exe -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed.exe -i "s/models_model_tflite/g_model/g" models/model.cc

In [ ]:
sound1 = AudioSegment.from_file("sound1.wav")

In [22]:
CHUNK = 1024

wf_z1 = wave.open("ding.wav", 'rb')

p_z1 = pyaudio.PyAudio()
stream_z1 = p_z1.open(format=p_z1.get_format_from_width(wf_z1.getsampwidth()),
                channels=wf_z1.getnchannels(),
                rate=wf_z1.getframerate(),
                output=True)


def start_play():
    wf_z1.setpos(0)
    for i in np.arange(1,16000/CHUNK):
        data = wf_z1.readframes(CHUNK)
        stream_z1.write(data)
    
start_play()
# stream.stop_stream()
# stream.close()
# p.terminate()




    

### 语音采样

In [114]:
WAVE_OUTPUT_FILENAME="speech_commands_v0.01/guandeng/"
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 16000
RECORD_SECONDS = 1  #需要录制的时间

p = pyaudio.PyAudio()	#初始化



stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                    frames_per_buffer=CHUNK)#创建录音文件
   


def start_audio():
 
   
    all_frames=[]
    for j in range(0,10):
        frames = []
        print(j,end='   ')
#        延迟0.6s
        for i in range(0,10):
            stream.read(CHUNK)

        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)+1):
            data = stream.read(CHUNK)
            frames.append(data)#开始录音
           
        all_frames.append(frames)
  
    for f in all_frames:
        WAVE_OUTPUT_FILENAME ="speech_commands_v0.01/test/" + str(int(round(time.time() * 1000)))  + ".wav"
        wf = wave.open("speech_commands_v0.01/test/temp.wav", 'wb')	#保存
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)    
        wf.writeframes(b''.join(f))
        wf.close()
        AudioSegment.from_file("speech_commands_v0.01/test/temp.wav").set_channels(1).export(WAVE_OUTPUT_FILENAME, format="wav")

    os.remove("speech_commands_v0.01/test/temp.wav")    
    
start_audio()

stream.stop_stream()
stream.close()
p.terminate()

0   1   2   3   4   5   6   7   8   9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64   65   66   67   68   69   70   71   72   73   74   75   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96   97   98   99   

In [20]:
CHUNK = 1024

wf = wave.open("ding.wav", 'rb')

p = pyaudio.PyAudio()
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

print(wf)
data = wf.readframes(CHUNK)


while data !=b'':
    stream.write(data)
    data = wf.readframes(CHUNK)

stream.stop_stream()
stream.close()
p.terminate()

NameError: name 'wave' is not defined

In [ ]:


def start_audio(time = 3,save_file="test.wav"):
	CHUNK = 1024
	FORMAT = pyaudio.paInt16
	CHANNELS = 2
	RATE = 16000
	RECORD_SECONDS = time  #需要录制的时间
	WAVE_OUTPUT_FILENAME = save_file	#保存的文件名

	p = pyaudio.PyAudio()	#初始化
	print("ON")

	stream = p.open(format=FORMAT,
	                channels=CHANNELS,
	                rate=RATE,
	                input=True,
		                frames_per_buffer=CHUNK)#创建录音文件
	frames = []

	for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
	    data = stream.read(CHUNK)
	    frames.append(data)#开始录音

	print("OFF")

	stream.stop_stream()
	stream.close()
	p.terminate()

	wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')	#保存
	wf.setnchannels(CHANNELS)
	wf.setsampwidth(p.get_sample_size(FORMAT))
	wf.setframerate(RATE)
	wf.writeframes(b''.join(frames))
	wf.close()

start_audio()

In [139]:
# playsound('ding.wav')

In [ ]:
import time,threading

from datetime import datetime

from PIL import ImageGrab

from cv2 import *

import numpy as np

from pynput import keyboard



def video_record(): # 录入视频

    global name

    name = datetime.now().strftime('%Y-%m-%d %H-%M-%S') # 当前的时间（当文件名）

    screen = ImageGrab.grab() # 获取当前屏幕

    width, high = screen.size # 获取当前屏幕的大小

    fourcc = VideoWriter_fourcc('X', 'V', 'I', 'D') # MPEG-4编码,文件后缀可为.avi .asf .mov等

    video = VideoWriter('%s.avi' % name, fourcc, 15, (width, high)) # （文件名，编码器，帧率，视频宽高）

    #print('3秒后开始录制----') # 可选

    #time.sleep(3)

    print('开始录制!')

    global start_time

    start_time = time.time()

    while True:

        if flag:

        print("录制结束！")

        global final_time

        final_time = time.time()

        video.release() #释放

        break

        im = ImageGrab.grab() # 图片为RGB模式

        imm = cvtColor(np.array(im), COLOR_RGB2BGR) # 转为opencv的BGR模式

        video.write(imm) #写入

# time.sleep(5) # 等待5秒再次循环

def on_press(key): # 监听按键

    global flag

    if key == keyboard.Key.home:

    flag = True # 改变

    return False # 返回False，键盘监听结束！

def video_info(): # 视频信息

    video = VideoCapture('%s.avi' % name) # 记得文件名加格式不要错！

    fps = video.get(CAP_PROP_FPS)

    Count = video.get(CAP_PROP_FRAME_COUNT)

    size = (int(video.get(CAP_PROP_FRAME_WIDTH)), int(video.get(CAP_PROP_FRAME_HEIGHT)))

    print('帧率=%.1f'%fps)

    print('帧数=%.1f'%Count)

    print('分辨率',size)

    print('视频时间=%.3f秒'%(int(Count)/fps))

    print('录制时间=%.3f秒'%(final_time-start_time))

    print('推荐帧率=%.2f'%(fps*((int(Count)/fps)/(final_time-start_time))))

if __name__ == '__main__':

    flag = False

    th = threading.Thread(target=video_record)

    th.start()

with keyboard.Listener(on_press=on_press) as listener:

    listener.join()

    time.sleep(1) # 等待视频释放过后

    video_info()